In [2]:
from datetime import datetime
from sqlalchemy import create_engine, Column, String, DateTime, Date, JSON
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session, sessionmaker

Base = declarative_base()

class StockAction(Base):
    __tablename__ = 'stock_actions'
    
    id = Column(String(50), primary_key=True)
    data = Column(JSON)  # Store all the nested data as JSON
    date = Column(Date)
    name = Column(String)
    create_time = Column(DateTime, default=datetime.now)
    update_time = Column(DateTime)

def init_db():
    engine = create_engine('postgresql://localhost:5432/leek_commune')
    Base.metadata.create_all(engine)
    return sessionmaker(bind=engine)()

# Example usage:
"""
action = StockAction(
    id=str(uuid.uuid4()),
    data={
        'action_field': {
            'id': '49489bfde5eb439e9eabb74c00048611',
            'name': '公告',
            'date': '2024-11-28',
            'status': 0
        },
        'stock': {
            'code': 'sh605136',
            'name': '丽人丽妆'
        },
        # ... other nested data
    }
)
"""


/var/folders/s8/wsqkwqzx6vd6__fhz4sbfh5m0000gp/T/ipykernel_69978/481321436.py:6: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


"\naction = StockAction(\n    id=str(uuid.uuid4()),\n    data={\n        'action_field': {\n            'id': '49489bfde5eb439e9eabb74c00048611',\n            'name': '公告',\n            'date': '2024-11-28',\n            'status': 0\n        },\n        'stock': {\n            'code': 'sh605136',\n            'name': '丽人丽妆'\n        },\n        # ... other nested data\n    }\n)\n"

In [66]:
from IPython.display import JSON

import requests
import json



url = 'https://app.jiuyangongshe.com/jystock-app/api/v1/action/field'

headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.9',
    'content-type': 'application/json',
    'cookie': 'SESSION=YTZjZmMxYTUtNTYxOS00NzIzLWI3NzktM2QwNTA1ZjIwNzFk; Hm_lvt_58aa18061df7855800f2a1b32d6da7f4=1732592522,1733474230; Hm_lpvt_58aa18061df7855800f2a1b32d6da7f4=1733718713',
    'origin': 'https://www.jiuyangongshe.com',
    'platform': '3',
    'priority': 'u=1, i',
    'referer': 'https://www.jiuyangongshe.com/',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'timestamp': '1733738548404',
    'token': '6c1ad68e6fb7ca578e5fe527fd2de413',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'
}

data = {
    'date': '2024-12-10',
    'pc': 1
}

response = requests.post(url, headers=headers, json=data)

print(response.json)

<bound method Response.json of <Response [200]>>


In [68]:
session = init_db()

In [32]:
#response.json()

In [70]:
stock_data = response.json()['data']
valid_data = []
print(stock_data[0])
for datum in stock_data:
    item = {}
    item['data'] = datum
    action_field_date = datum.get('date')
    if datum.get('action_field_id') != '':
        exists = session.query(StockAction).filter_by(id=datum.get('action_field_id')).first() != None
        if not exists:
            item['id'] = datum.get('action_field_id')  # Set id to action_field id
            item['date'] = datetime.strptime(action_field_date, '%Y-%m-%d').date()
            valid_data.append(item)

# Bulk insert only valid data
actions = [StockAction(**item) for item in valid_data]
session.bulk_save_objects(actions)
session.commit()

{'date': '2024-12-10', 'reason': '', 'action_field_id': '', 'name': '简图', 'count': 0}


In [72]:
print(stock_data)

[{'date': '2024-12-10', 'reason': '', 'action_field_id': '', 'name': '简图', 'count': 0}, {'action_field_id': 'b054f5284ddb4c2881077ab389f23f58', 'name': '公告', 'date': '2024-12-10', 'reason': '', 'status': 0, 'sort_no': 0, 'is_delete': '0', 'delete_time': None, 'create_time': '2024-12-10 11:56:03', 'update_time': None, 'count': 3, 'list': [{'code': 'sh603356', 'name': '华菱精工', 'article': {'article_id': '4tzc7n9ljs', 'comment_count': 2, 'like_count': 2, 'create_time': '2024-12-10 11:56:03', 'user_id': '1', 'is_like': 0, 'action_info': {'article_id': '4tzc7n9ljs', 'action_info_id': '0bc8e60a21c24f579cd52258c82b8561', 'stock_id': 'ead2d3a8fddf4c1abc28fa34fa0c6b00', 'action_field_id': 'b054f5284ddb4c2881077ab389f23f58', 'time': '09:25:02', 'num': None, 'price': 1615, 'day': None, 'edition': None, 'shares_range': 1001.0, 'reason': None, 'expound': '实控人拟变更+光伏支架+风电+电梯\n1、今日复牌。2024年12月9日盘后公告，公司拟向上饶巨准启鸣企业管理合伙定增募资不超4.53亿，本次发行完成后，公司控股股东由黄业华变更为上饶巨准启鸣企业管理合伙，公司实际控制人为由黄业华家族变更为郑剑波。\n2、参股子公司安徽华菱新能源，该公司tpo